In [8]:
# def remove_headers():
#     for i in range(1,41):
#         for j in range(1,11):
#             with open(f"test/data/orl_faces/s{i}/{j}.pgm", "rb") as f:
#                 lines = f.readlines()
            
#             with open(f"test/data/processed_orl_faces/s{i}/{j}.pgm", "wb") as processed_f:
#                 processed_f.writelines(lines[3:])


In [22]:
from compression.helpers import read_data
import numpy as np

In [18]:
def get_images():
    images = []
    for i in range(1,41):
        person_images = []
        for j in range(1,11):
            person_images.append(read_data(f"test/data/orl_faces/s{i}/{j}.pgm")[0])
        images.append(person_images)
    return images

In [19]:
images = get_images()

In [20]:
len(images)

40

In [26]:
np.random.choice(10)

8

## Cross Validation

In [36]:
def cross_validation(images):
    training_images = []
    test_images = []

    for i in range(0,len(images)):
        test_index = np.random.choice(10)
        training_images.append(images[i].pop(test_index))
        test_images.append(images[i])
    return training_images, test_images

## Training

In [ ]:
def train(images, dicts, k):
    for i, person_images in enumerate(images):
        for image in person_images:
            for j in range(0, k):
                dicts[i][j] += image[j]

## Testing

In [ ]:
# Instanciates LZW 
from compression import LZW
lzw = LZW(dictionary_k={}) # needs required dict

In [38]:
def test(test_images, dicts, k, verbose: bool) -> float:

    accuracy_result = []
    
    for index, image in enumerate(test_images):
        compression_size = []

        for index in range(40):
            compression_length, _ = lzw.encode() # needs required arguments

            compression_size.append(compression_length)


        accuracy_result.append(index  == compression_size.index(min(compression_size)))
        accuracy = 100 * accuracy_result.count(True) / len(accuracy_result)

        if verbose:
            print(f"Modelo com acurácia para k = {k}: {accuracy}%")
    return accuracy 

## Running

In [ ]:
def train_test():
    pass